# PokerBot Championship Pipeline (Google Colab)

This notebook sets up and runs the full PokerBot championship pipeline in Colab:
1. Clone the GitHub repository
2. Install dependencies
3. Generate training data
4. Train the model (GPU)
5. Validate results

**Note:** Runtime should be set to GPU for best performance.

In [ ]:
# 1. Clone the PokerBot GitHub repository
!git clone https://github.com/elliotttmiller/pokerbot.git
%cd pokerbot

In [ ]:
# 2. Install required dependencies
!pip install -r requirements.txt

In [ ]:
# 3. Generate new training data (15 min for 1K samples)
!python scripts/generate_quick_data.py --samples 1000 --cfr-iters 2000

In [ ]:
# 4. Train with championship config (30 min on GPU)
!python scripts/train_deepstack.py --config scripts/config/championship.json --use-gpu --epochs 200

In [ ]:
# 5. Validate results
!python scripts/validate_deepstack_model.py --model models/versions/best_model.pt

## Results and Next Steps

- Review the output of the validation cell for model quality and recommendations.
- For deeper analysis, run `scripts/run_analysis_report.py` to generate plots and metrics in `models/reports`.
- To retrain or tune, adjust config or sample size and rerun relevant cells.

**Tip:** If you encounter missing dependencies, install them with `!pip install <package>` as needed.

In [ ]:
# 6. Run analysis report and display results
!python scripts/run_analysis_report.py

# Display generated plots inline (optional)
import matplotlib.pyplot as plt
from IPython.display import Image, display

for plot in [
    "models/reports/loss_curves.png",
    "models/reports/correlation_curves.png",
    "models/reports/per_street_correlation.png",
    "models/reports/per_bucket_corrs_hist.png"
]:
    try:
        display(Image(plot))
    except Exception:
        print(f"Plot not found: {plot}")

In [ ]:
# Error Handling and Logging Example
import traceback

def run_step(cmd, log_file):
    import subprocess
    try:
        result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        with open(log_file, 'w') as f:
            f.write(result.stdout.decode())
            f.write(result.stderr.decode())
        print(f"Step succeeded: {cmd}")
    except Exception as e:
        with open(log_file, 'w') as f:
            f.write(traceback.format_exc())
        print(f"Step failed: {cmd}\nSee {log_file} for details.")

# Example usage:
run_step('python scripts/train_deepstack.py --config scripts/config/championship.json --use-gpu --epochs 200', 'train_log.txt')

In [ ]:
# Interactive Visualization Example
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# Plot training/validation loss curves interactively
metrics_path = 'models/reports/training_metrics.csv'
if os.path.exists(metrics_path):
    df = pd.read_csv(metrics_path)
    plt.figure(figsize=(10,5))
    plt.plot(df[df['phase']=='train']['epoch'], df[df['phase']=='train']['loss'], label='Train Loss')
    plt.plot(df[df['phase']=='valid']['epoch'], df[df['phase']=='valid']['loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()
    plt.show()
else:
    print('training_metrics.csv not found.')

# Plot per-bucket correlations interactively
corr_path = 'models/reports/per_bucket_corrs.json'
if os.path.exists(corr_path):
    import json
    with open(corr_path) as f:
        data = json.load(f)
    bucket_corrs = np.array(data.get('bucket_corrs', []))
    plt.figure(figsize=(8,4))
    plt.hist(bucket_corrs, bins=30, alpha=0.8)
    plt.title('Per-Bucket Correlations')
    plt.xlabel('Correlation')
    plt.ylabel('Count')
    plt.show()
else:
    print('per_bucket_corrs.json not found.')

In [ ]:
# Model Download Cell
import shutil
from google.colab import files

# Zip models and reports for download
shutil.make_archive('pokerbot_artifacts', 'zip', 'models')
files.download('pokerbot_artifacts.zip')